# Mach-Zehnder Interferometer (MZI) ve Matematiksel Model

## Çıkış Yoğunlukları

Bir MZI'nin çıkış portlarındaki ışık yoğunlukları faz farkına bağlı olarak şu şekilde ifade edilir:

$$
I_{\text{output1}} = I_{\text{input}} \cdot \cos^2\left(\frac{\Delta \phi}{2}\right)
$$

$$
I_{\text{output2}} = I_{\text{input}} \cdot \sin^2\left(\frac{\Delta \phi}{2}\right)
$$

Burada:

-   \( I\_{\text{input}} \): Giriş ışığının yoğunluğu
-   \( \Delta \phi = \phi_1 - \phi_2 \): Faz farkı

---

## MZI'nin Matematiksel Temsili

Bir MZI'nin unitary matris dönüşümü şu şekilde ifade edilir:

$$
U = e^{i\gamma}
\begin{bmatrix}
e^{i\phi_1} & 0 \\
0 & e^{i\phi_2}
\end{bmatrix}
\begin{bmatrix}
\cos\theta & -\sin\theta \\
\sin\theta & \cos\theta
\end{bmatrix}
$$

Burada:

-   \( \phi_1, \phi_2 \): Faz kaydırıcıların ayarları
-   \( \theta \): Beam splitter'ın bölme oranı (fotonların iki çıkış portuna nasıl dağıldığını belirler)
-   \( \gamma \): Genel bir faz faktörü (sistem içindeki global faz)

---

## Fiziksel Model Akışı

Bir MZI'nin fiziksel yapısı şu şekilde gösterilebilir:

1. **Giriş:** Işık, giriş portundan sisteme girer.
2. **Birinci Beam Splitter:** Işık iki kola bölünür.
3. **Faz Kaydırıcılar:** Her kolda faz kaydırıcılar bulunur ve bu kaydırıcılar ışığın fazını \( \phi_1 \) ve \( \phi_2 \) kadar değiştirir.
4. **İkinci Beam Splitter:** Kollardan gelen ışık ikinci beam splitter üzerinde birleştirilir.
5. **Çıkış:** Çıkış portlarındaki ışık yoğunlukları girişim deseni oluşturur.

Bu sürecin fiziksel akışı şu şekilde ifade edilebilir:


![MZI Meshes](img/Illustration-of-the-most-common-MZI-mesh-topologies-for-implementing-unitary-matrix.jpg)


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from neurophox.numpy import RMNumpy

ImportError: cannot import name 'typing' from 'numpy' (c:\Users\cumon\Desktop\Neurophox\torch_env\lib\site-packages\numpy\__init__.py)

In [3]:
# 1. MNIST Dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
])

In [4]:
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

In [5]:
# 2. Define Optical Layer with Neurophox
class OpticalLayer(nn.Module):
    def __init__(self, input_size, output_size):
        super(OpticalLayer, self).__init__()
        # Neurophox Rectangular Mesh
        self.mesh = RMNumpy(input_size, theta_init_name="haar_rect")
        self.output_size = output_size

    def forward(self, x):
        # Propagate through the optical mesh
        x = x.detach().numpy()  # Convert to NumPy for Neurophox
        output = torch.tensor(self.mesh.propagate(x).squeeze())
        return output

In [6]:
# 3. Define Full Model
class MNISTOpticalModel(nn.Module):
    def __init__(self):
        super(MNISTOpticalModel, self).__init__()
        self.flatten = nn.Flatten()
        self.optical_layer = OpticalLayer(784, 784)  # 784 input and output ports
        self.fc = nn.Linear(784, 10)  # Fully connected layer for classification

    def forward(self, x):
        x = self.flatten(x)  # Flatten 28x28 image to 784
        x = self.optical_layer(x)  # Pass through optical mesh
        x = self.fc(x)  # Fully connected layer
        return x

In [7]:
# 4. Training Loop
def train_model(model, train_loader, criterion, optimizer, epochs=5):
    for epoch in range(epochs):
        model.train()
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

In [8]:
# 5. Test Loop
def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Accuracy: {100 * correct / total:.2f}%")

In [9]:
# 6. Initialize Model, Loss, and Optimizer
model = MNISTOpticalModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [10]:
# 7. Train and Test
train_model(model, train_loader, criterion, optimizer, epochs=5)
test_model(model, test_loader)

Şeyma
Şeyma
Şeyma


MemoryError: Unable to allocate 601. MiB for an array with shape (785, 64, 784) and data type complex128